In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import Flowers102
from torch.utils.data import DataLoader
import torchvision.models as models
import numpy as np

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the Flowers102 dataset with RGB images
train_dataset = Flowers102(
    root='./data',
    split='train',
    transform=transform,
    download=True
)

val_dataset = Flowers102(
    root='./data',
    split='val',
    transform=transform,
    download=True
)

test_dataset = Flowers102(
    root='./data',
    split='test',
    transform=transform,
    download=True
)

# Create data loaders for training, validation, and test sets
batch_size = 64  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize a pre-trained ResNet-18 model and move it to the GPU
feature_extractor = models.resnet18(pretrained=True)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2]).to(device)  # Remove the classification layer

# Define the prototype-based few-shot learner
class FewShotLearner(nn.Module):
    def __init__(self, feature_extractor, num_classes):
        super(FewShotLearner, self).__init__()
        self.feature_extractor = feature_extractor
        self.num_classes = num_classes
        self.out_size = self.calculate_output_size()
        self.prototypes = nn.Parameter(torch.randn(num_classes, self.out_size).to(device))

    def calculate_output_size(self):
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224).to(device)
            features = self.feature_extractor(dummy_input)
            return features.view(features.size(0), -1).size(1)

    def forward(self, x):
        x = x.to(device)
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        dists = torch.cdist(features, self.prototypes)
        return -dists

# Create a few-shot learner
num_classes = 102
few_shot_learner = FewShotLearner(feature_extractor, num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(few_shot_learner.parameters(), lr=0.001, momentum=0.9)

In [16]:
# Early stopping parameters
patience = 5  # Number of epochs with no improvement to wait
best_val_accuracy = 0.0
early_stopping_counter = 0

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    few_shot_learner.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = few_shot_learner(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    few_shot_learner.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = few_shot_learner(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print('Early stopping - no improvement in validation accuracy for {} epochs.'.format(patience))
        break

Epoch [1/100], Train Loss: 3.4521, Train Accuracy: 30.49%, Validation Loss: 4.3229, Validation Accuracy: 6.96%
Epoch [2/100], Train Loss: 2.7136, Train Accuracy: 65.20%, Validation Loss: 4.0851, Validation Accuracy: 11.47%
Epoch [3/100], Train Loss: 2.0786, Train Accuracy: 88.82%, Validation Loss: 3.9000, Validation Accuracy: 17.45%
Epoch [4/100], Train Loss: 1.5777, Train Accuracy: 97.45%, Validation Loss: 3.7423, Validation Accuracy: 22.55%
Epoch [5/100], Train Loss: 1.1804, Train Accuracy: 99.41%, Validation Loss: 3.6119, Validation Accuracy: 26.57%
Epoch [6/100], Train Loss: 0.8789, Train Accuracy: 100.00%, Validation Loss: 3.5197, Validation Accuracy: 30.20%
Epoch [7/100], Train Loss: 0.6673, Train Accuracy: 100.00%, Validation Loss: 3.4501, Validation Accuracy: 32.75%
Epoch [8/100], Train Loss: 0.5278, Train Accuracy: 100.00%, Validation Loss: 3.3923, Validation Accuracy: 33.63%
Epoch [9/100], Train Loss: 0.4249, Train Accuracy: 100.00%, Validation Loss: 3.3480, Validation Accura

In [17]:
# Test the model on the test set
few_shot_learner.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = few_shot_learner(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 3.0128, Test Accuracy: 44.36%


### Data Augmentation

In [18]:
# Define data augmentation transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),  # Randomly crop and resize
        transforms.RandomHorizontalFlip(),  # Randomly flip horizontally
        transforms.RandomRotation(30),  # Randomly rotate by up to 30 degrees
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
        transforms.RandomGrayscale(p=0.2),  # Convert to grayscale with a 20% probability
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Random perspective transformation
        transforms.RandomVerticalFlip(),  # Randomly flip vertically
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),  # Resize for validation and test
        transforms.CenterCrop(224),  # Center crop for validation and test
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),  # Resize for validation and test
        transforms.CenterCrop(224),  # Center crop for validation and test
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Apply the data augmentation transformations to your datasets
train_dataset = Flowers102(
    root='./data',
    split='train',
    transform=data_transforms['train'],  # Use data augmentation for training
    download=True
)

val_dataset = Flowers102(
    root='./data',
    split='val',
    transform=data_transforms['val'],  # Use validation data transformations
    download=True
)

test_dataset = Flowers102(
    root='./data',
    split='test',
    transform=data_transforms['test'],  # Use test data transformations
    download=True
)

# Create data loaders for training, validation, and test sets
batch_size = 64  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize a pre-trained ResNet-18 model and move it to the GPU
feature_extractor = models.resnet18(pretrained=True)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2]).to(device)  # Remove the classification layer

# Define the prototype-based few-shot learner
class FewShotLearner(nn.Module):
    def __init__(self, feature_extractor, num_classes):
        super(FewShotLearner, self).__init__()
        self.feature_extractor = feature_extractor
        self.num_classes = num_classes
        self.out_size = self.calculate_output_size()
        self.prototypes = nn.Parameter(torch.randn(num_classes, self.out_size).to(device))

    def calculate_output_size(self):
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224).to(device)
            features = self.feature_extractor(dummy_input)
            return features.view(features.size(0), -1).size(1)

    def forward(self, x):
        x = x.to(device)
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        dists = torch.cdist(features, self.prototypes)
        return -dists

# Create a few-shot learner
num_classes = 102
few_shot_learner = FewShotLearner(feature_extractor, num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(few_shot_learner.parameters(), lr=0.001, momentum=0.9)

In [19]:
# Early stopping parameters
patience = 5  # Number of epochs with no improvement to wait
best_val_accuracy = 0.0
early_stopping_counter = 0

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    few_shot_learner.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = few_shot_learner(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    few_shot_learner.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = few_shot_learner(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print('Early stopping - no improvement in validation accuracy for {} epochs.'.format(patience))
        break

Epoch [1/100], Train Loss: 5.0106, Train Accuracy: 0.59%, Validation Loss: 4.9233, Validation Accuracy: 1.08%
Epoch [2/100], Train Loss: 4.8420, Train Accuracy: 1.37%, Validation Loss: 4.8211, Validation Accuracy: 1.67%
Epoch [3/100], Train Loss: 4.7514, Train Accuracy: 1.57%, Validation Loss: 4.7090, Validation Accuracy: 2.06%
Epoch [4/100], Train Loss: 4.6535, Train Accuracy: 1.86%, Validation Loss: 4.5909, Validation Accuracy: 2.06%
Epoch [5/100], Train Loss: 4.5335, Train Accuracy: 2.84%, Validation Loss: 4.4778, Validation Accuracy: 4.02%
Epoch [6/100], Train Loss: 4.4414, Train Accuracy: 4.12%, Validation Loss: 4.3721, Validation Accuracy: 6.37%
Epoch [7/100], Train Loss: 4.3081, Train Accuracy: 6.67%, Validation Loss: 4.2518, Validation Accuracy: 8.33%
Epoch [8/100], Train Loss: 4.2044, Train Accuracy: 9.12%, Validation Loss: 4.1478, Validation Accuracy: 9.12%
Epoch [9/100], Train Loss: 4.0434, Train Accuracy: 14.22%, Validation Loss: 4.0279, Validation Accuracy: 12.55%
Epoch [1

Epoch [74/100], Train Loss: 1.1265, Train Accuracy: 79.80%, Validation Loss: 0.9453, Validation Accuracy: 80.29%
Epoch [75/100], Train Loss: 1.1724, Train Accuracy: 75.98%, Validation Loss: 0.9495, Validation Accuracy: 79.22%
Epoch [76/100], Train Loss: 1.1749, Train Accuracy: 75.49%, Validation Loss: 0.9413, Validation Accuracy: 79.71%
Epoch [77/100], Train Loss: 1.0950, Train Accuracy: 79.31%, Validation Loss: 0.9033, Validation Accuracy: 80.69%
Early stopping - no improvement in validation accuracy for 5 epochs.


In [20]:
# Test the model on the test set
few_shot_learner.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = few_shot_learner(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 0.9899, Test Accuracy: 78.61%


### Resize to (500,500) -> Edit first layer of resnet

In [24]:
# Define data augmentation transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((500, 500)),   # Randomly crop and resize
        transforms.RandomHorizontalFlip(),  # Randomly flip horizontally
        transforms.RandomRotation(30),  # Randomly rotate by up to 30 degrees
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
        transforms.RandomGrayscale(p=0.2),  # Convert to grayscale with a 20% probability
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation
        transforms.RandomPerspective(distortion_scale=0.5, p=0.5),  # Random perspective transformation
        transforms.RandomVerticalFlip(),  # Randomly flip vertically
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((500, 500)),  # Resize for validation and test
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((500, 500)),  # Resize for validation and test
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

# Apply the data augmentation transformations to your datasets
train_dataset = Flowers102(
    root='./data',
    split='train',
    transform=data_transforms['train'],  # Use data augmentation for training
    download=True
)

val_dataset = Flowers102(
    root='./data',
    split='val',
    transform=data_transforms['val'],  # Use validation data transformations
    download=True
)

test_dataset = Flowers102(
    root='./data',
    split='test',
    transform=data_transforms['test'],  # Use test data transformations
    download=True
)

# Create data loaders for training, validation, and test sets
batch_size = 64  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize a pre-trained ResNet-18 model and move it to the GPU
feature_extractor = models.resnet18(pretrained=True)
# Modify the first convolutional layer to accept 3-channel (RGB) images of size 500x500
# You need to change the in_channels argument to 3 and kernel_size to 7
feature_extractor.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
feature_extractor = nn.Sequential(*list(feature_extractor.children())[:-2]).to(device)  # Remove the classification layer

# Define the prototype-based few-shot learner
class FewShotLearner(nn.Module):
    def __init__(self, feature_extractor, num_classes):
        super(FewShotLearner, self).__init__()
        self.feature_extractor = feature_extractor
        self.num_classes = num_classes
        self.out_size = self.calculate_output_size()
        self.prototypes = nn.Parameter(torch.randn(num_classes, self.out_size).to(device))

    def calculate_output_size(self):
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 500, 500).to(device)
            features = self.feature_extractor(dummy_input)
            return features.view(features.size(0), -1).size(1)

    def forward(self, x):
        x = x.to(device)
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)
        dists = torch.cdist(features, self.prototypes)
        return -dists

# Create a few-shot learner
num_classes = 102
few_shot_learner = FewShotLearner(feature_extractor, num_classes)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(few_shot_learner.parameters(), lr=0.001, momentum=0.9)

In [25]:
# Early stopping parameters
patience = 5  # Number of epochs with no improvement to wait
best_val_accuracy = 0.0
early_stopping_counter = 0

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
    few_shot_learner.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = few_shot_learner(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    few_shot_learner.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = few_shot_learner(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print('Early stopping - no improvement in validation accuracy for {} epochs.'.format(patience))
        break

Epoch [1/100], Train Loss: 5.0164, Train Accuracy: 1.27%, Validation Loss: 5.0336, Validation Accuracy: 0.98%
Epoch [2/100], Train Loss: 4.9266, Train Accuracy: 1.18%, Validation Loss: 4.9019, Validation Accuracy: 1.67%
Epoch [3/100], Train Loss: 4.8395, Train Accuracy: 1.47%, Validation Loss: 4.8679, Validation Accuracy: 1.27%
Epoch [4/100], Train Loss: 4.7731, Train Accuracy: 2.16%, Validation Loss: 4.7925, Validation Accuracy: 2.55%
Epoch [5/100], Train Loss: 4.7336, Train Accuracy: 1.18%, Validation Loss: 4.7609, Validation Accuracy: 2.06%
Epoch [6/100], Train Loss: 4.6496, Train Accuracy: 2.06%, Validation Loss: 4.6849, Validation Accuracy: 3.04%
Epoch [7/100], Train Loss: 4.5925, Train Accuracy: 1.57%, Validation Loss: 4.6285, Validation Accuracy: 3.53%
Epoch [8/100], Train Loss: 4.5013, Train Accuracy: 3.63%, Validation Loss: 4.5531, Validation Accuracy: 3.63%
Epoch [9/100], Train Loss: 4.4736, Train Accuracy: 3.63%, Validation Loss: 4.4854, Validation Accuracy: 4.12%
Epoch [10/

In [26]:
# Test the model on the test set
few_shot_learner.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = few_shot_learner(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

Test Loss: 2.5585, Test Accuracy: 44.61%
